In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import time 
import torch
from torch import Tensor, empty
from typing import Optional, Tuple, Union, List
import helpers_functional as F
from helpers_layer import *

### Testing of Conv2d Layer

In [51]:
N = 1000
in_channels = 4
out_channels = 10
kernel_size = (2, 3)
padding = (2,3)
stride = 2

conv_pytorch = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
conv = Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
x = torch.randn((N, in_channels, 32, 32))

t1 = time.time()
dummy = conv_pytorch(x)
t2 = time.time()
expected = conv(x)
t3 = time.time()
actual = F.conv2d(x, conv.weight, conv.bias, padding=padding, stride=stride)
t4 = time.time()
actual2 = F.conv2d(x, conv_pytorch.weight, conv_pytorch.bias, padding=padding, stride=stride)

print("Expected.shape:", expected.shape)
print("Actual.shape:", actual.shape)

torch.testing.assert_allclose(actual, expected)
torch.testing.assert_allclose(actual2, dummy)

print("\nComputation Time Comparison:")
print("\tPytorch Conv2d: \t", t2-t1)
print("\tImplemented Conv2d: \t", t3-t2)
print("\tConv2d: \t\t", t4-t3)

Expected.shape: torch.Size([1000, 10, 18, 18])
Actual.shape: torch.Size([1000, 10, 18, 18])

Computation Time Comparison:
	Pytorch Conv2d: 	 0.02079010009765625
	Implemented Conv2d: 	 0.03800606727600098
	Conv2d: 		 0.03204202651977539


### Testing of ConvTranspose2d Layer

In [52]:
N = 100
in_channels = 4
out_channels = 10
kernel_size = (6, 9)
dilation = (2,2)
padding = (2,0)
output_padding = 1
stride = 1

convtranspose_pytorch = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, output_padding=output_padding, dilation=dilation)
convtranspose = ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, output_padding=output_padding, dilation=dilation)
x = torch.randn((N, in_channels, 32, 32))

t1 = time.time()
dummy = convtranspose_pytorch(x)
print("Dummy.shape:", dummy.shape)
t2 = time.time()
expected = convtranspose(x)
print("Expected.shape:", expected.shape)
t3 = time.time()
actual = F.conv_transpose2d(x, convtranspose.weight, convtranspose.bias, padding=padding, stride=stride, dilation=dilation, output_padding=output_padding)
print("Actual.shape:", actual.shape)
t4 = time.time()
actual2 = F.conv_transpose2d(x, convtranspose_pytorch.weight, convtranspose_pytorch.bias, padding=padding, stride=stride, dilation=dilation, output_padding=output_padding)


torch.testing.assert_allclose(actual, expected)
torch.testing.assert_allclose(actual2, dummy)

print("\nComputation Time Comparison:")
print("\tPytorch Conv2d: \t", t2-t1)
print("\tImplemented Conv2d: \t", t3-t2)
print("\tConv2d: \t\t", t4-t3)

Dummy.shape: torch.Size([100, 10, 39, 49])
Expected.shape: torch.Size([100, 10, 39, 49])
Actual.shape: torch.Size([100, 10, 39, 49])

Computation Time Comparison:
	Pytorch Conv2d: 	 0.1132659912109375
	Implemented Conv2d: 	 0.18238592147827148
	Conv2d: 		 0.17034912109375


### Testing of Backward of Conv2d

In [53]:
N = 1000
in_channels = 4
out_channels = 10
kernel_size = (2, 3)
padding = (2,3)
stride = 2

conv_pytorch = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
conv = Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
x = torch.randn((N, in_channels, 32, 32))

dummy = conv_pytorch(x)
expected = conv(x)
actual = F.conv2d(x, conv.weight, conv.bias, padding=padding, stride=stride)
actual2 = F.conv2d(x, conv_pytorch.weight, conv_pytorch.bias, padding=padding, stride=stride)

print("Expected.shape:", expected.shape)

torch.testing.assert_allclose(actual, expected)
torch.testing.assert_allclose(actual2, dummy)

grad_wrt_output = torch.ones_like(expected, requires_grad=True)
print(grad_wrt_output.shape)

#expected_bward_out = conv_pytorch.backward(grad_wrt_output)
#print("Expected Backward.shape:", expected_bward_out.shape)





Expected.shape: torch.Size([1000, 10, 18, 18])
torch.Size([1000, 10, 18, 18])


In [54]:
with torch.no_grad():
    input = torch.normal(mean = torch.zeros(1,4,8,8),std = 1)

    kernel = torch.tensor([4,4])
    stride = 1
    padding = 0
    dilation= 1
    groups = 1
    in_channels = 4
    out_channels = 10

    conv = torch.nn.Conv2d(4, 2, kernel, stride=stride, padding=padding, dilation=dilation, groups=groups)
    weight = conv.state_dict()["weight"]
    bias = conv.state_dict()["bias"]

    conv_ = Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)

    m = conv.forward(input) 
    print("size",m.size())
    m_ = conv_.forward(input)


grad_output = torch.normal(mean=torch.zeros(m.size(0), m.size(1), m.size(-2), m.size(-1)), std= 1)
print(grad_output.shape)

res = torch.nn.functional.grad.conv2d_weight(input, weight.shape, grad_output, stride = stride, dilation=dilation)

print(res.shape)

with torch.no_grad():
    grad = conv_.backward(grad_output)
print(grad.size())

input.requires_grad_(True)
weight.requires_grad_(True)
    


size torch.Size([1, 2, 5, 5])
torch.Size([1, 2, 5, 5])
torch.Size([2, 4, 4, 4])


ValueError: requested an input grad size of [8, 8], but valid sizes range from [6, 7] to [6, 7] (for a grad_output of torch.Size([5, 5]))